In [1]:
import pickle
import spacy
import en_core_web_md
import collections
import pandas as pd
import nltk

In [2]:
with open('preprocessing.pickle', 'rb') as f:
    combine_df = pickle.load(f)

In [3]:
train_df = pd.read_csv('train_tweets.csv')

In [4]:
test_df = pd.read_csv('test_tweets.csv')

In [5]:
train_df.shape[0] + test_df.shape[0] == combine_df.shape[0]

True

In [6]:
combine_df.head(2)

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thank, lyft, credit, use, cause, offer, wheel..."


### Тема “Создание признакового пространства”

Используем предобработанные в рамках 1-ого домашнего задания датасет combine_df_prepocessed.pkl. Используем столбец 'clean_tweet'.

Задание 1.  
Используя библиотеку Spacy, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? (Учтите, что max_word_limit_spacy для Spacy = 1000000)  


In [7]:
nlp = en_core_web_md.load()

In [8]:
string_combine = combine_df.clean_tweet.to_string()
string_train = combine_df.clean_tweet[:train_df.shape[0]].to_string()
string_test = combine_df.clean_tweet[train_df.shape[0]:].to_string()

In [9]:
nlp.max_length = len(string_combine)

In [10]:
%%time
doc = nlp(string_combine)
doc_train = nlp(string_train)
doc_test = nlp(string_test)

CPU times: user 1min 41s, sys: 21 s, total: 2min 2s
Wall time: 2min 1s


In [11]:
labels = []
for e in doc.ents:
    labels.append((e.label_, e.text))

In [12]:
ner = collections.Counter()

In [13]:
for word in labels:
    ner[word[0]] += 1

In [14]:
ner.most_common(20)

[('CARDINAL', 41827),
 ('DATE', 9672),
 ('PERSON', 5725),
 ('GPE', 3492),
 ('ORG', 2272),
 ('TIME', 1372),
 ('NORP', 908),
 ('ORDINAL', 512),
 ('FAC', 391),
 ('PRODUCT', 323),
 ('LOC', 169),
 ('EVENT', 145),
 ('QUANTITY', 113),
 ('LAW', 64),
 ('WORK_OF_ART', 43),
 ('PERCENT', 35),
 ('MONEY', 35),
 ('LANGUAGE', 25)]

#### С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных.   Действительно ли в топ вошли только персоны и организации или есть мусор?

In [15]:
person_train = collections.Counter()
person_test = collections.Counter()
organization_train = collections.Counter()
organization_test = collections.Counter()

In [16]:
for e in doc_train.ents:
    if e.label_ == 'ORG':
        organization_train[e.text] += 1
    elif e.label_ == 'PERSON':
        person_train[e.text] += 1

In [17]:
for en in doc_test.ents:
    if en.label_ == 'ORG':
        organization_test[en.text] += 1
    elif en.label_ == 'PERSON':
        person_test[en.text] += 1

In [18]:
dd = pd.DataFrame()
dd['popular_organization_train'] = organization_train.most_common(20)
dd['popular_organization_test'] = organization_test.most_common(20)
dd['popular_person_train'] = person_train.most_common(20)
dd['popular_person_test'] = person_test.most_common(20)

In [19]:
dd.head(20)

,popular_organization_train,popular_organization_test,popular_person_train,popular_person_test
0,"(gop, 33)","(nba, 10)","(conde, 26)","(obama, 20)"
1,"(nba, 22)","(gop, 9)","(hillary, 19)","(feminismisterrorism feminismm, 18)"
2,"(cavs, 20)","(islam, 9)","(christina grimmie, 18)","(suppo, 13)"
3,"(social analytics, 14)","(cavs, 7)","(obama, 17)","(fath, 11)"
4,"(un, 13)","(un, 6)","(feminismisterrorism feminismm, 17)","(sta, 9)"
5,"(islam, 12)","(starbucks, 6)","(bjp, 17)","(lebron, 8)"
6,"(cnn, 11)","(fbi, 5)","(sta, 16)","(jo cox, 8)"
7,"(ios, 9)","(sirf ashiq log, 5)","(bea, 14)","(bjp, 8)"
8,"(fami, 8)","(congress, 5)","(fath, 13)","(christina grimmie, 7)"
9,"(fed, 8)","(bbc, 4)","(tom, 12)","(lo, 6)"


## Задание 2.  
Используя библиотеку nltk, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? Для данного задания используем ограничение на количество символов во входном датасете (max_word_limit_spacy = 1000000), чтобы иметь возможность сравнить результаты работы Spacy и nltk. Обратите внимание, что nltk чувствителен к регистру.  
С помощью nltk выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?

In [20]:
string_combine_ = combine_df.tweet.to_string()

In [21]:
string_combine_tokenize = nltk.word_tokenize(string_combine_)

In [22]:
string_combine_tags = nltk.pos_tag(string_combine_tokenize)

In [23]:
string_combine_chunk = nltk.ne_chunk (string_combine_tags)

In [24]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(string_combine))) if hasattr(chunk, 'label') }

set()

Никаких сущностей, к сожалению,  nltk не нашел